In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [2]:
num_variaveis = 2
w = [1, 2, 3, 4]
confianca = 0.75
X = [f'X {i+1}' for i in range(num_variaveis)]
Z = [f'Z {i+1}' for i in range(len(w))]
pi = [1 / len(w)] * len(w)
pi = pd.DataFrame(pi, index = w)

M = 3.5
model = ConcreteModel('Big M')

#Sets e Parâmetros
model.w = RangeSet(len(w))
model.pi = Param(model.w, initialize = pi)
model.confianca = Param(initialize = confianca)
model.M = Param(initialize = M)
model.i = RangeSet(num_variaveis)  # Para x_1 e x_2


# Variáveis
model.x = Var(model.i, domain=NonNegativeReals)
model.Z = Var(model.w, domain=Binary)

#Restrições
def restricao1(model, omega):
    return omega * model.x[1] + model.x[2] >= 7 - M * (1 - model.Z[omega])
model.restricao = Constraint(model.w, rule=restricao1)

def restricao2(model):
    return sum(model.Z[j] * model.pi[j] for j in model.w) >= model.confianca
model.restricao2 = Constraint(rule=restricao2)

# Função objetivo
model.obj = Objective(expr=sum(model.x[i] for i in model.i), sense=minimize)

# Resolvendo o modelo
solver = SolverFactory('glpk')
result = solver.solve(model)



In [3]:
# Exibindo os resultados
print("x_1:", model.x[1].value)
print("x_2:", model.x[2].value)
for k in model.w:
    print(f"Z_{k}:", model.Z[k].value)

x_1: 3.5
x_2: 0.0
Z_1: 0.0
Z_2: 1.0
Z_3: 1.0
Z_4: 1.0


**1)O que acontece se alterar o nível de confiança?**

As variáveis binárias de Z serão ativadas conforme o aumento da confiança, e serão desativadas com a diminuição.
Isso impacta nos valores das variáveis também. Com o aumento da confiança a variável x1 se aproxima de 7.

**2)O que acontece se o valor de M não for adequado?**
Com valores de M superiores a 3.5, X1 retornará 3.5 normalmente, porém com valores inferiores a 3.5 a variável X1 se aproxima de 7